In [1]:
from __future__ import division, print_function, absolute_import

%matplotlib inline

import os

import matplotlib.pyplot as plt
import astropy.io.ascii as at
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
import numpy as np

from hypra.utils import cat_io

home_dir = os.getenv("HOME")

In [2]:
hdat, _, _, _ = cat_io.get_data("H")
hpos = SkyCoord(hdat["RA"],hdat["DEC"],unit=u.degree)

In [3]:
rotators = np.where((hdat["PERIOD"]>0))[0]

In [4]:
c13file = os.path.join(home_dir,"HyPra/catalogs/K2Campaign13targets_ours.csv")
c13 = at.read(c13file)

In [5]:
print(c13.dtype)

[('EPIC ID', '<i8'), ('RA (J2000) [deg]', '<f8'), ('Dec (J2000) [deg]', '<f8'), ('magnitude', '<f8'), ('Investigation IDs', 'S153')]


In [6]:
c13pos = SkyCoord(c13["RA (J2000) [deg]"],c13["Dec (J2000) [deg]"],unit=u.degree)

In [7]:
idx,sep,_ = c13pos.match_to_catalog_sky(hpos)

In [8]:
good_match = np.where(sep<(5*u.arcsec))[0]

good_idx = idx[good_match]

In [9]:
print(hdat["EPIC_ID"][good_idx])

[    -9999     -9999     -9999     -9999     -9999     -9999     -9999
 210640966 210651981     -9999     -9999 210667985 210674207 210709514
 210718930     -9999     -9999 210741091 210742017 210743724 210754620
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -9999     -9999     -9999     -9999     -9999     -9999     -9999
     -

In [10]:
np.union1d?

In [13]:
of_interest_all = np.union1d(rotators,good_idx)

bright_only = np.where(hdat["KH_MASS"]>0.3)[0]

of_interest = np.intersect1d(of_interest_all,bright_only)

print(len(of_interest))

156


In [18]:
htab = Table(hdat)
print(htab.dtype.names)

('RA', 'DEC', 'ASAS_ID', 'HIP_ID', 'KUNDERT_MATCH', 'KUNDERT_RA', 'KUNDERT_DEC', 'KUNDERT_PROT', 'KUNDERT_OBS', 'KUNDERT_SPT', 'ROESER_MATCH', 'ROESER_PMRA', 'ROESER_PMRA_ERR', 'ROESER_PMDEC', 'ROESER_PMDEC_ERR', 'ROESER_PARALLAX', 'ROESER_PARALLAX_ERR', 'ROESER_DISTANCE', 'ROESER_RV', 'ROESER_VPAR', 'ROESER_VPAR_ERR', 'ROESER_VPER', 'ROESER_VPER_ERR', 'ROESER_MASS', 'TWOMASSNAME', 'TWOMASS_RA', 'TWOMASS_DEC', 'TWOMASS_J', 'TWOMASS_JERR', 'TWOMASS_H', 'TWOMASS_HERR', 'TWOMASS_K', 'TWOMASS_KERR', 'TWOMASS_FLAG', 'TWOMASS_PHOT_FLAG', 'TWOMASS_SEP_OPT', 'TWOMASS_B', 'TWOMASS_VR', 'DELORME_MATCH', 'SWASP_ID', 'USNO_ID', 'DELORME_RA', 'DELORME_DEC', 'DELORME_XRAY', 'DELORME_V', 'DELORME_PREV_SPT', 'DELORME_PMEM', 'DELORME_LITP', 'DELORME_DELTACORE', 'DELORME_PARALLAX', 'DELORME_DISTANCE', 'ANYPERIOD', 'UCAC_MATCH', 'UCAC_RA', 'UCAC_DEC', 'UCAC_ID', 'UCAC_FLAG', 'UCAC_DBLSTAR', 'UCAC_PMRA', 'UCAC_PMDEC', 'UCAC_J', 'UCAC_JERR', 'UCAC_H', 'UCAC_HERR', 'UCAC_K', 'UCAC_KERR', 'UCAC_B', 'UCAC_BER

In [19]:
ra_str = np.zeros(len(hdat),"S15")
dec_str = np.zeros(len(hdat),"S15")

for i in of_interest:
    skypos = hpos[i].to_string("hmsdms",sep=":")
    ra_str[i], dec_str[i] = skypos.split()
print(ra_str[of_interest][:10])
print(dec_str[of_interest][:10])

['04:27:35.8275' '04:20:12.8958' '04:26:05.8601' '04:27:46.0091'
 '04:37:31.9292' '04:24:12.398' '04:50:33.7629' '04:24:28.2573'
 '04:28:48.232' '04:06:16.0557']
['+15:35:21.2976' '+19:14:00.8255' '+15:31:27.5999' '+11:44:11.2183'
 '+15:08:47.3927' '+14:45:29.8391' '+15:05:00.6133' '+16:53:10.4147'
 '+17:17:07.9346' '+15:41:53.4044']


In [20]:
hprint = htab["RA","DEC","PERIOD"]
hprint["IDX"] = np.arange(len(hdat),dtype="int")
hprint["RA_HMS"] = ra_str
hprint["DEC_DMS"] = dec_str

In [21]:
formats = {"RA":"%.6f","DEC":"%.6f","PERIOD":"%.2f"}
at.write(hprint["IDX","RA","DEC","RA_HMS","DEC_DMS","PERIOD"][of_interest],
         "hyades_rotators_and_C13targets2.csv",delimiter=",",
         formats=formats,overwrite=True)